<img src="../Img/Python_SQL_PI.png" style="width:100%">


<p style="font-size:22px;text-align:center">Лекция 1. Работа с СУБД SQLite с помощью Python</p> 

24 февраля 2022 года<br>
Поток: ПИ20-1, ПИ20-2, ПИ20-3<br>

28 февраля 2022 года<br>
Поток: ПИ20-4, ПИ20-5, ПИ20-6<br>


<a id = Home></a>

<p style="font-size:18px;text-align:left">Учебные вопросы</p>

1. [Работа с базами данных в Python](#1)
2. [Создание базы данных SQLite из программы Python.](#2)
3. [Создание таблиц в базе данных SQLite с помощью Python.](#3)
4. [Вставка данных в базу данных SQLite с помощью Python.](#4)
5. [Обновление данных в базе данных SQLite с помощью Python.](#5)
6. [Удаление данных в базе данных SQLite с помощью Python.](#6)
7. [Запросы к данным базы данных SQLite с помощью Python.](#7)

<a id=1></a> [К оглавлению](#Home)

### 1. Работа с базами данных в Python

Популярным инструментами для работы с базами данных в Python являются:

- <a href="https://pypi.python.org/pypi/pysqlite">PySQLite и модуль sqlite3</a>
- <a href="https://rogerbinns.github.io/apsw/">APSW</a>
- <a href="https://www.sqlalchemy.org/">SQLAlchemy</a>

На этом занятии изучим работу с базами данных с помощью модуля *sqlite3*.

<a id=2></a>[К оглавлению](#Home)

### 2. Создание базы данных SQLite из программы Python

Покажем, как создать новую базу данных SQLite из программы Python с помощью модуля *sqlite3*.

Когда мы подключаемся к несуществующему файлу базы данных SQLite, SQLite автоматически создает для нас новую базу данных. Чтобы создать базу данных, сначала нам нужно создать объект Connection с помощью функции `connect()` модуля sqlite3. Например, следующая программа Python создает новый файл базы данных *pythonsqlite.db* в указанной папке. Перед запуском программы эту папку необходимо создать. Если путь не указывать, файл БД будет создан в текущей папке.

In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

In [3]:
if __name__ == '__main__':
    create_connection("python_sqlite.db") # Создание файла БД в текущей папке
    create_connection(r"D:\FinU\tmp\sample_sqlite.db") # создание файла БД в указанной папке

2.6.0
2.6.0


Во-первых, мы определяем функцию с именем `create_connection()`, которая подключается к базе данных SQLite, указанной в файле базы данных `db_file`. Внутри функции мы вызываем функцию `connect()` модуля sqlite3.

Функция `connect()` открывает соединение с базой данных SQLite. Он возвращает объект `Connection`, представляющий базу данных. Используя объект `Connection`, вы можете выполнять различные операции с базой данных.

В случае возникновения ошибки мы перехватываем ее в блоке `try except` и отображаем сообщение об ошибке. Если все в порядке, мы показываем версию базы данных SQLite.

Хорошей практикой программирования является завершение соединения с базой данных, если она больше не используется.

Во-вторых, мы передаем путь к файлу базы данных функции `create_connection()` для создания базы данных. Обратите внимание, что префикс `r` в `r"C:\sqlite\db\pythonsqlite.db"` указывает Python, что мы передаем необработанную строку.

Существует возможность создания файла БД в оператиной памяти компьютера. Для этого параметр `:memory:` функции `Connect()` вместо имени файла.

In [4]:
create_connection(":memory:")

2.6.0


<a id=3></a> [К оглавлению](#Home)

### 3. Создание таблиц в базе данных SQLite с помощью Python

Покажем, как создавать таблицы в базе данных SQLite из программы Python.

Чтобы создать новую таблицу в базе данных SQLite из программы Python, выполним следующие действия:

1. Создадим объект *Connection*, используя функцию `connect()` модуля *sqlite3*.
2. Создадим объект *Cursor*, вызвав метод `cursor()` объекта *Connection*.
3. Передадим оператор `CREATE TABLE` методу `execute()` объекта *Cursor* и выполним этот метод.

Создадим две таблицы: проекты и задачи, как показано на следующей диаграмме:

<img src="../Img/Python-SQLite-Sample-Database.jpg" style="width:50%">

Для создания такой БД потребуется следующий код на языке SQL:

```
-- projects table
CREATE TABLE IF NOT EXISTS projects (
	id integer PRIMARY KEY,
	name text NOT NULL,
	begin_date text,
	end_date text
);

-- tasks table
CREATE TABLE IF NOT EXISTS tasks (
	id integer PRIMARY KEY,
	name text NOT NULL,
	priority integer,
	project_id integer NOT NULL,
	status_id integer NOT NULL,
	begin_date text NOT NULL,
	end_date text NOT NULL,
	FOREIGN KEY (project_id) REFERENCES projects (id)
);
```

Создадим эти таблицы с помощью Python, следуя шагам вышеописанного алгоритма.

In [5]:
create_table_projects = """
CREATE TABLE IF NOT EXISTS projects (
    id integer PRIMARY KEY,
    name text NOT NULL,
    begin_date text,
    end_date text
); """

create_table_tasks = """
CREATE TABLE IF NOT EXISTS tasks (
    id integer PRIMARY KEY,
    name text NOT NULL,
    priority integer,
    project_id integer NOT NULL,
    status_id integer NOT NULL,
    begin_date text NOT NULL,
    end_date text NOT NULL,
    FOREIGN KEY (project_id) REFERENCES projects (id)
);
"""

In [6]:
conn = sqlite3.connect("python_sqlite.db") # 1
crs = conn.cursor() # 2
crs.execute(create_table_projects) # 3
crs.execute(create_table_tasks)

Чтобы убедиться, что таблицы созданы, откроем базу данных *python_sqlite.db* в коммандном окне *sqlite3* и выполним команду `.tables`.

<img src="../Img/sqlite3_command_line_1.png" style="width:40%">

<a id=4></a> [К оглавлению](#Home)

### 4. Вставка данных в базу данных SQLite с помощью Python

Изучим этапы вставки данных в таблицы базы данных SQLite с помощью Python. Выполним следующие действия:

1. Подключимся к базе данных SQLite, создав объект Connection.
2. Создадим объект Cursor, вызвав метод курсора объекта Connection.
3. Выполним оператор INSERT. Чтобы передать аргументы оператору INSERT, используем вопросительный знак (?) в качестве заполнителя для каждого аргумента.

Создадим проект и несколько заданий.

In [7]:
# Функция для создания соединения с базой данных
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

# Функция для добавления проекта
def create_project(conn, project):
    """
    Создаёт проект, возвращает идентификатор проекта
    """
    sql = ''' INSERT INTO projects(name,begin_date,end_date)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

# Функция для добавления заданий проекта
def create_task(conn, task):
    """
    Создание задания
    """
    sql = ''' INSERT INTO tasks(name,priority,project_id,status_id,begin_date,end_date)
              VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid

# Основная функция
def main():
    database = "python_sqlite.db"

    # Создание соединения с базой данных
    conn = create_connection(database)
    with conn:
        # Создание нового проекта
        project = ('Cool App with SQLite & Python', '2022-02-10', '2022-02-28');
        project_id = create_project(conn, project)

        # Задания
        task_1 = ('Analyze the requirements of the app', 1, project_id, 1, '2022-02-10', '2022-02-15')
        task_2 = ('Confirm with user about the top requirements', 1, project_id, 1, '2022-02-15', '2022-02-28')

        # create tasks
        create_task(conn, task_1)
        create_task(conn, task_2)

In [8]:
if __name__ == '__main__':
    main()

Подключимся к базе данных *python_sqlite.db* с помощью интерфейса командной строки и проверим вставку записей проекта и заданий, выполнив sql-запросы:

```
select * from projects;
select * from tasks;
```

<img src="../Img/sqlite3_command_line_2.png" style="width:100%">

<a id=5></a> [К оглавлению](#Home)

### 5. Обновление данных

В этом разделе научимся с помощью Python обновлять данные в созданной ранее базе данных. Выполним следующие шаги алгоритма обновления данных.

1. Подключимся к базе данных SQLite, создав объект *Connection*.
2. Создадим объект *Cursor*, вызвав метод `cursor()` объекта *Connection*.
3. Выполним команду `UPDATE` с помощью вызова метода `execute()` объекта *Cursor*.

Изменим приоритет, начальную и конечную даты одного задания. Создадим функцию обновления данных.

In [9]:
def update_task(conn, task):
    """
    обновляет приоритет, начальную дату, конечную дату задания
    """
    sql = ''' UPDATE tasks
              SET priority = ? ,
                  begin_date = ? ,
                  end_date = ?
              WHERE id = ?'''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()

Создадим соединение и передадим объект соединения в функцию обновления.

In [10]:
conn = create_connection("python_sqlite.db")
update_task(conn, (2, '2022-02-21', '2022-02-25', 2))

Убедимся, что изменения произошли с помощью интерфейса командной строки.

<img src="../Img/sqlite3_command_line_3.png" style="width:100%">

<a id=6></a> [К оглавлению](#Home)

### 6. Удаление данных

В этом разделе научимся с помощью Python удалять данные в созданной ранее базе данных. Выполним следующие шаги алгоритма удаления данных.

1. Подключимся к базе данных SQLite, создав объект *Connection*.
2. Создадим объект *Cursor*, вызвав метод `cursor()` объекта *Connection*.
3. Выполним команду `DELETE` с помощью вызова метода `execute()` объекта *Cursor*. При необходимости передачи аргументов в метод *execute()* используем знаки вопроса (?).

Удалим одно из заданий. Создадим для этого функцию удаления задания, одним из параметров функции станет идентификатор удаляемого задания. Также создадим функцию удаления всех заданий.

In [11]:
def delete_task(conn, id):
    """
    Удаляет задание по идентификатору
    """
    sql = 'DELETE FROM tasks WHERE id=?'
    cur = conn.cursor()
    cur.execute(sql, (id,))
    conn.commit()

def delete_all_tasks(conn):
    """
    Удаляет все задания
    """
    sql = 'DELETE FROM tasks'
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

Выполним шаги алгоритма по удалению задания 2.

In [12]:
conn = create_connection("python_sqlite.db")
delete_task(conn, 2)

Проверим результат с помощью интерфейса командной строки, запросив все записи таблицы *tasks* с помощью sql-запроса

```
select * from tasks;
```

<img src="../Img/sqlite3_command_line_4.png" style="width:100%">

Мы убедились, что в таблице заданий осталось только одно задание.

<a id=7></a> [К оглавлению](#Home)

### 7. Запросы к данным

В этом разделе научимся удалять данные из базы данных SQLite с помощью Python.

Для выполнения запроса к базе данных с помошщью Python выполним шаги следующего алгоритма.

1. Подключимся к базе данных SQLite, создав объект *Connection*.
2. Создадим объект *Cursor*, вызвав метод `cursor()` объекта *Connection*.
3. Вполним команду SELECT.
4. Выполним метод `fetchall()` курсора для получения данных.
5. Пройдем в цикле по элементам курсора и обработаем каждую строку отдельно.

Прежде, чем выполнять программы этого раздела, вернём строку со вторым заданием в таблицу *tasks* с помощью кода из раздела 3 "Вставка данных в базу данных SQLite с помощью Python".

<img src="../Img/Python-SQLite-Sample-Database.jpg" style="width:50%">

In [13]:
conn = create_connection("python_sqlite.db")
task_2 = ('Confirm with user about the top requirements', 2, 1, 1, '2022-02-15', '2022-02-28')
create_task(conn, task_2)

2

Создадим функцию, которая отбирает все записи таблицы заданий и функцию, отбирающую задания по приеритету.

In [14]:
def select_all_tasks(conn):
    """
    Запрашивает все записи таблицы заданий

    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM tasks")

    rows = cur.fetchall()

    for row in rows:
        print(row)

def select_task_by_priority(conn, priority):
    """
    Отбирает задания по приоритету
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM tasks WHERE priority=?", (priority,))

    rows = cur.fetchall()

    for row in rows:
        print(row)

В функции *select_task_by_priority()* мы выбрали задачи на основе определенного приоритета. Знак вопроса (?) в запросе является символом подстановки (заполнителем). Выполняя команду SELECT, курсор заменяет символ подстановки параметром функции. Метод `fetchall()` извлекает все найденные задачи.

функция main() создаёт соединение и вызывает функцию запроса.

In [15]:
def main():
    database = "python_sqlite.db"

    # create a database connection
    conn = create_connection(database)
    with conn:
        print("1. Query task by priority:")
        select_task_by_priority(conn, 1)

        print("2. Query all tasks")
        select_all_tasks(conn)

In [16]:
main()

1. Query task by priority:
(1, 'Analyze the requirements of the app', 1, 1, 1, '2022-02-10', '2022-02-15')
2. Query all tasks
(1, 'Analyze the requirements of the app', 1, 1, 1, '2022-02-10', '2022-02-15')
(2, 'Confirm with user about the top requirements', 2, 1, 1, '2022-02-15', '2022-02-28')
